In [1]:
import os
from openai import OpenAI

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [2]:
import requests

SYSTEM_PROMPT = f"""당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.
실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)
추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.
당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.
"""

MESSAGES = [
    {
        'role': 'system',
        'content': SYSTEM_PROMPT
    }
]

def recommend(query_text):
    url = "http://localhost:8000/recommend"
    response = requests.post(url, json={"query_text": query_text})
    return response.json()


tools = [
    {
        "type": "function",
        "function": {
            "name": "recommend",
            "description": "사용자 발화 기반으로 메뉴 추천 API를 호출합니다. 오로지 이 함수 결과로만 메뉴 추천되어야 합니다. 현재 해장 또는 다이어트 2개 카테고리에 대한 메뉴 추천만 가능하며, 사용자 발화가 없는 경우 빈 리스트가 반환 될 수 있습니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query_text": {
                        "type": "string",
                        "description": "사용자 발화 텍스트 원문",
                    },
                },
                "required": ["query_text"],
                "additionalProperties": False,
            },
        }
    }
]

In [3]:
MESSAGES.append(
    {"role": "user", "content": "안녕"}
)

completion = client.chat.completions.create(
    messages=MESSAGES,
    model="gpt-4o-2024-08-06",
    tools=tools
)

completion

ChatCompletion(id='chatcmpl-A2Vi3WQb249ZigEmgZ7EhZFPuQZYK', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='안녕하세요! 무엇을 도와드릴까요? 맛있는 메뉴 추천이 필요하신가요?', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1725161911, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_9e15ccd6a4', usage=CompletionUsage(completion_tokens=23, prompt_tokens=245, total_tokens=268))

In [4]:
if completion.choices[0].finish_reason == "tool_calls":
    print('function calling lets go')
else:
    MESSAGES.append(
        {"role": "assistant", "content": completion.choices[0].message.content}
    )

In [5]:
MESSAGES

[{'role': 'system',
  'content': "당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.\n실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)\n추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.\n당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.\n"},
 {'role': 'user', 'content': '안녕'},
 {'role': 'assistant', 'content': '안녕하세요! 무엇을 도와드릴까요? 맛있는 메뉴 추천이 필요하신가요?'}]

In [6]:
MESSAGES.append(
    {"role": "user", "content": "해장 메뉴 추천해줄레?"}
)

completion = client.chat.completions.create(
    messages=MESSAGES,
    model="gpt-4o-2024-08-06",
    tools=tools
)

completion

ChatCompletion(id='chatcmpl-A2Vi3sI697KFHmS0kwxYsD9VNOTMX', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_BBq6RxyI6q9SCV0YLN2Ufhwp', function=Function(arguments='{"query_text":"해장"}', name='recommend'), type='function')]))], created=1725161911, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_cbac5eb3c0', usage=CompletionUsage(completion_tokens=16, prompt_tokens=283, total_tokens=299))

In [7]:
import json

if completion.choices[0].finish_reason == "tool_calls":
    tool_calls = completion.choices[0].message.tool_calls
    tool_name = tool_calls[0].function.name
    tool_args = tool_calls[0].function.arguments
    tool_id = tool_calls[0].id
    print('function calling lets go')
else:
    MESSAGES.append(
        {"role": "assistant", "content": completion.choices[0].message.content}
    )

function calling lets go


In [8]:
completion.choices[0].message.tool_calls[0]

ChatCompletionMessageToolCall(id='call_BBq6RxyI6q9SCV0YLN2Ufhwp', function=Function(arguments='{"query_text":"해장"}', name='recommend'), type='function')

In [9]:
tool_result = recommend(**json.loads(tool_args))
tool_result[:3]

[{'_id': '1244448_12',
  'score': 0.4079955859860546,
  'menu': '뽕따프라페',
  'restaurant': '플러스82-분당점',
  'url': 'https://www.yogiyo.co.kr/mobile/#/1244448'},
 {'_id': '1244448_11',
  'score': 0.3963077914410869,
  'menu': '수박주스x3',
  'restaurant': '플러스82-분당점',
  'url': 'https://www.yogiyo.co.kr/mobile/#/1244448'},
 {'_id': '1244448_1',
  'score': 0.39073353386951637,
  'menu': '미숫가루',
  'restaurant': '플러스82-분당점',
  'url': 'https://www.yogiyo.co.kr/mobile/#/1244448'}]

In [10]:
tool_calls[0].id

'call_BBq6RxyI6q9SCV0YLN2Ufhwp'

In [11]:
MESSAGES

[{'role': 'system',
  'content': "당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.\n실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)\n추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.\n당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.\n"},
 {'role': 'user', 'content': '안녕'},
 {'role': 'assistant', 'content': '안녕하세요! 무엇을 도와드릴까요? 맛있는 메뉴 추천이 필요하신가요?'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄레?'}]

In [12]:
MESSAGES.append({
    "role": "assistant",  # user does not work
    "content": None,
    "tool_calls": [
        {
            "id": tool_id,
            "type": "function",
            "function": {
                "name": tool_name,
                "arguments": tool_args
            }
        }
    ]
})

In [13]:
tool_args

'{"query_text":"해장"}'

In [14]:
MESSAGES

[{'role': 'system',
  'content': "당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.\n실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)\n추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.\n당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.\n"},
 {'role': 'user', 'content': '안녕'},
 {'role': 'assistant', 'content': '안녕하세요! 무엇을 도와드릴까요? 맛있는 메뉴 추천이 필요하신가요?'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄레?'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_BBq6RxyI6q9SCV0YLN2Ufhwp',
    'type': 'function',
    'function': {'name': 'recommend', 'arguments': '{"query_text":"해장"}'}}]}]

In [15]:
tool_result[:3]

[{'_id': '1244448_12',
  'score': 0.4079955859860546,
  'menu': '뽕따프라페',
  'restaurant': '플러스82-분당점',
  'url': 'https://www.yogiyo.co.kr/mobile/#/1244448'},
 {'_id': '1244448_11',
  'score': 0.3963077914410869,
  'menu': '수박주스x3',
  'restaurant': '플러스82-분당점',
  'url': 'https://www.yogiyo.co.kr/mobile/#/1244448'},
 {'_id': '1244448_1',
  'score': 0.39073353386951637,
  'menu': '미숫가루',
  'restaurant': '플러스82-분당점',
  'url': 'https://www.yogiyo.co.kr/mobile/#/1244448'}]

In [25]:
MESSAGES.append({
    "role": "tool",
    "content": json.dumps(tool_result[:3], ensure_ascii=False),
    "tool_call_id": tool_id
})

In [26]:
MESSAGES

[{'role': 'system',
  'content': "당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.\n실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)\n추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.\n당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.\n"},
 {'role': 'user', 'content': '안녕'},
 {'role': 'assistant', 'content': '안녕하세요! 무엇을 도와드릴까요? 맛있는 메뉴 추천이 필요하신가요?'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄레?'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_BBq6RxyI6q9SCV0YLN2Ufhwp',
    'type': 'function',
    'function': {'name': 'recommend', 'arguments': '{"query_text":"해장"}'}}]},
 {'role': 'tool',
  'content': '[{"_id": "1244448_12", "score": 0.4079955859860546, "menu": "뽕따프라페", "restaurant": "플러스82-분당점", "url": "https://www.yogiyo.co.kr/mobile/#/1244448"}, {"_id": "1244448_11", "score": 0.3963077914410869, "menu": "수박주스x3", "restaurant": "플러스82-분당점", "url": "https://www.yogiyo.co.kr/mobile/#/1244

In [27]:
completion = client.chat.completions.create(
    messages=MESSAGES,
    model="gpt-4o-2024-08-06",
    tools=tools
)
completion

ChatCompletion(id='chatcmpl-A2VkhklHTXk5ZadwVzpPE526sU0Vo', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='해장을 위한 메뉴를 추천해드릴게요:\n\n1. [뽕따프라페 - 플러스82-분당점](https://www.yogiyo.co.kr/mobile/#/1244448)\n2. [수박주스x3 - 플러스82-분당점](https://www.yogiyo.co.kr/mobile/#/1244448)\n3. [미숫가루 - 플러스82-분당점](https://www.yogiyo.co.kr/mobile/#/1244448)\n\n각 메뉴는 소화와 해장에 도움을 줄 수 있는 음료들이에요. 이 메뉴들로 하루를 상쾌하게 시작해보세요!', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1725162075, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_9e15ccd6a4', usage=CompletionUsage(completion_tokens=148, prompt_tokens=506, total_tokens=654))

In [31]:
MESSAGES.append(
    {"role": "assistant", "content": completion.choices[0].message.content}
)

In [32]:
MESSAGES

[{'role': 'system',
  'content': "당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.\n실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)\n추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.\n당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.\n"},
 {'role': 'user', 'content': '안녕'},
 {'role': 'assistant', 'content': '안녕하세요! 무엇을 도와드릴까요? 맛있는 메뉴 추천이 필요하신가요?'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄레?'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_BBq6RxyI6q9SCV0YLN2Ufhwp',
    'type': 'function',
    'function': {'name': 'recommend', 'arguments': '{"query_text":"해장"}'}}]},
 {'role': 'tool',
  'content': '[{"_id": "1244448_12", "score": 0.4079955859860546, "menu": "뽕따프라페", "restaurant": "플러스82-분당점", "url": "https://www.yogiyo.co.kr/mobile/#/1244448"}, {"_id": "1244448_11", "score": 0.3963077914410869, "menu": "수박주스x3", "restaurant": "플러스82-분당점", "url": "https://www.yogiyo.co.kr/mobile/#/1244